# Imports

In [134]:
from glob import glob
from pybaseball import playerid_reverse_lookup
import pandas as pd
import numpy as np

# Functions

In [123]:
def gather_hits(df, pitch_types, hit_types):
    """
    df : <Pandas DataFrame Object>
    pitch_types : <List> Contains all the columns for the different pitch types.
    hit_types : <List> Contains all of the columns to store the whether or not a hit occured
        for some pitch type.
    
    Removes pitches from the players record where the pitch resulted in a ball.
    Hit_type records whether or not the pitch for a specific pitch_type resulted in a hit.
    Foul balls are not considered to be a succesful hit.
    
    returns : <Pandas DataFrame Object> Updated DataFrame.
    """
    for pitch_type, hit_type in zip(pitch_types, hit_types):
        # Only count a pitch against a player if it wasn't a ball, because otherwise its the pitcher's fault.
        df[pitch_type] = ((df[pitch_type] == 1) & ( ~df["description"].str.contains("ball"))) * 1
        df[hit_type] = ((df[pitch_type] == 1) & (df["description"].str.contains("hit"))) * 1
        
    return df

def calculate_percentage_hit(df, pitch_types, hit_types):
    """
    df : <Pandas DataFrame Object>
    pitch_types : <List> Contains all the columns for the different pitch types.
    hit_types : <List> Contains all of the columns to store the whether or not a hit occured
        for some pitch type.
    
    Calculates percentage hit chance for some pitch type. (hit_type / pitch_type) * 100
    Removes NaN values by replacing them with zero. A NaN value my occur if a player never
        was pitched a pitch_type (0/0 error).
    
    returns : <Pandas DataFrame Object> Updated DataFrame.
    """
    for pitch_type, hit_type in zip(pitch_types, hit_types):
        df[pitch_type.replace("pitch_type", "percentage_hit")] = (df[hit_type] / df[pitch_type]) * 100
    
    df = df.fillna(0)
    return df

def calculate_specialty_score(df, percentage_hit_types, weights):
    """
    df : <Pandas DataFrame Object>
    percentage_hit_types : <List> The column names containing percentage score for each pitch type.
    weights : <Dict> The corresponding weights for each pitch type.
    
    Sums up all of the percentage hits and applies their corresponding weights to get a specialty score
        statistic.
    
    returns : <Pandas DataFrame Object> Updated DataFrame.
    """
    df["speciality_score"] = 0
    for percentage_hit_type in percentage_hit_types:
        df["speciality_score"] += df[percentage_hit_type] * weights[percentage_hit_type]
    
    return df

def load_data(data_path):
    """
    data_path : <str> Path containing csv files with each season's batting data. Ex: ./data/*
    
    returns : <Pandas DataFrame Object> DataFrame containing concatenated data across all seasons
        found within 'data_path'.
    """
    all_data = [pd.read_csv(file_name, index_col=0) for file_name in glob(data_path)]
    return pd.concat(all_data, axis=0, sort=False)

# Loading all Statcast data

In [ ]:
data_path = "./data/*"
combined_data = load_data(data_path)

# Data Processing

In [148]:
# Remove all instances where the pitch type was not recorded.
n = combined_data.dropna(axis=0, subset=["pitch_type"])

# Below removes random pitch types that consist of numbers that look like 1623_12312 and only occur once.
# This line wil give a warning. Just ignore it.
n["pitch_type"] = n["pitch_type"].apply(lambda x: x.replace("_",""))
n = n[~(n["pitch_type"].str.isdigit())]

# Add columns with a 1 column indicating pitch type.
n = pd.get_dummies(n, columns=["pitch_type"])

# Add column to sum to get number of plate appearances.
n["plate_appearance"] = 1

# Add player info to the batting data, matching on the batting id (key_mlbam).
n["key_mlbam"] = n["batter"]
player_info = playerid_reverse_lookup(n["batter"].values, key_type='mlbam')
n = pd.merge(n, player_info,on="key_mlbam", how="inner")
n["season"] = n["game_year"]

pitch_types = ['CH', 'CU', 'EP', 'FC', 'FF', 'FO', 'FS', 'FT', 'IN', 'KC', 'KN', 'PO', 'SC', 'SI', 'SL', 'UN']
pitch_types = ["pitch_type_" + pitch_type for pitch_type in pitch_types]
hit_types = [pitch_type.replace("pitch_type", "hit") for pitch_type in pitch_types]

z = gather_hits(n.copy(), pitch_types, hit_types)

pitch_type_aggregates = {pitch_type:"sum" for pitch_type in pitch_types}
hit_type_aggregates = {hit_type:"sum" for hit_type in hit_types}
aggregates = {"name_last":"first", "name_first":"first", "season":"first", "plate_appearance":"sum", **pitch_type_aggregates, **hit_type_aggregates}

z = z.groupby(["batter", "game_year"]).agg(aggregates)

# Only get players with 200 or more plate appearances.
z = z[z["plate_appearance"] >= 200]

percentage_hit = calculate_percentage_hit(z.copy(), pitch_types, hit_types)

percentage_hit_types = [pitch_type.replace("pitch_type", "percentage_hit") for pitch_type in pitch_types]
weights = {'percentage_hit_CH': 1, 'percentage_hit_CU': 1, 'percentage_hit_EP': 1, 
           'percentage_hit_FC': 1, 'percentage_hit_FF': 1, 'percentage_hit_FO': 1, 
           'percentage_hit_FS': 1, 'percentage_hit_FT': 1, 'percentage_hit_IN': 1, 
           'percentage_hit_KC': 1, 'percentage_hit_KN': 1, 'percentage_hit_PO': 1, 
           'percentage_hit_SC': 1, 'percentage_hit_SI': 1, 'percentage_hit_SL': 1, 
           'percentage_hit_UN': 1}

specialty_score = calculate_specialty_score(percentage_hit.copy(), percentage_hit_types, weights)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Gathering player lookup table. This may take a moment.


In [149]:
specialty_score

name_last name_first  season  plate_appearance  \
batter   game_year                                                    
112526.0 2016.0          colon    bartolo  2016.0               232   
120074.0 2016.0          ortiz      david  2016.0              2472   
121347.0 2016.0      rodriguez       alex  2016.0               915   
134181.0 2016.0         beltre     adrian  2016.0              2442   
         2017.0         beltre     adrian  2017.0              1574   
         2018.0         beltre     adrian  2018.0              1857   
136860.0 2016.0        beltran     carlos  2016.0              2186   
         2017.0        beltran     carlos  2017.0              2014   
150029.0 2016.0          werth     jayson  2016.0              2785   
         2017.0          werth     jayson  2017.0              1405   
150229.0 2016.0     pierzynski      a. j.  2016.0               822   
276519.0 2016.0        rollins      jimmy  2016.0               679   
285079.0 2017.0         dickey      r. a.  2017.0               212   
346874.0 2016.0          uribe       juan  2016.0               933   
400085.0 2016.0         suzuki     ichiro  2016.0              1448   
         2017.0         suzuki     ichiro  2017.0               865   
400121.0 2016.0       martinez     victor  2016.0              2290   
         2017.0       martinez     victor  2017.0              1652   
         2018.0       martinez     victor  2018.0              1894   
400284.0 2016.0          utley      chase  2016.0              2224   
         2017.0          utley      chase  2017.0              1398   
         2018.0          utley      chase  2018.0               738   
405395.0 2016.0         pujols     albert  2016.0              2512   
         2017.0         pujols     albert  2017.0              2478   
         2018.0         pujols     albert  2018.0              1757   
407781.0 2016.0           byrd     marlon  2016.0               493   
407793.0 2016.0         lackey       john  2016.0               235   
         2017.0         lackey       john  2017.0               222   
407812.0 2016.0       holliday       matt  2016.0              1563   
         2017.0       holliday       matt  2017.0              1738   
...                        ...        ...     ...               ...   
656669.0 2017.0         luplow     jordan  2017.0               313   
         2018.0         luplow     jordan  2018.0               437   
656775.0 2018.0        mullins     cedric  2018.0               769   
656794.0 2018.0        newcomb       sean  2018.0               210   
656811.0 2018.0        o'hearn       ryan  2018.0               698   
656941.0 2017.0      schwarber       kyle  2017.0              2190   
         2018.0      schwarber       kyle  2018.0              2101   
657077.0 2018.0        verdugo       alex  2018.0               298   
657557.0 2017.0         dejong       paul  2017.0              1736   
         2018.0         dejong       paul  2018.0              1871   
657656.0 2018.0       laureano      ramon  2018.0               739   
660162.0 2017.0        moncada       yoan  2017.0               970   
         2018.0        moncada       yoan  2018.0              2634   
660271.0 2018.0         ohtani     shohei  2018.0              1441   
660670.0 2018.0          acuna     ronald  2018.0              2049   
663656.0 2018.0         tucker       kyle  2018.0               278   
664023.0 2017.0           happ        ian  2017.0              1655   
         2018.0           happ        ian  2018.0              1869   
664040.0 2018.0           lowe    brandon  2018.0               581   
664056.0 2017.0          bader   harrison  2017.0               365   
         2018.0          bader   harrison  2018.0              1766   
664057.0 2017.0      stevenson     andrew  2017.0               251   
         2018.0      stevenson     andrew  2018.0               343   
664058.0 2018.0       fletcher      david  20